## identify a repeat unit

In [1]:
import pgr
import networkx as nx
import os
import numpy as np
from collections import Counter 
#%matplotlib inline
import matplotlib.pyplot as plt

from bokeh.io import curdoc, show, output_notebook
from bokeh.models import Circle, ColumnDataSource, Grid, LinearAxis, Plot
from bokeh.plotting import figure
from bokeh.models import CrosshairTool
output_notebook()

Loading BokehJS ...

In [2]:

#sdb0 = pgr.SeqDB("/wd/LPA_HGRP_v2_seq_full_length.fa")
sdb0 = pgr.SeqDB("/home/yw2245/project_pi_hc895/yw2245/projects/lpa_genotyping/data/openhgl/HPRC/LPA_fa/LPA_HPRC_liftoff_all.fa")


sdb0.load_sequences()
sdb0.build_shmmrs_parallel(80, 56, 4)

In [4]:
#s0=sdb0.get_seq_by_name("grch38_chr6:160511303-160684379")

In [5]:
shmap = pgr.generate_shmmr_map(sdb0, sdb0, 1024)
mqr = pgr.map_interval_query(shmap, 0, 0, sdb0.get_len_by_id(0))
mc = pgr.find_match_chain(mqr,32)

In [6]:
orintation_count = [ ( (_[0],_[3]), _[6]) for _ in mc ]

In [7]:
orintation_count[:4]

[((0, 0), 0), ((0, 0), 0), ((0, 0), 0), ((0, 0), 0)]

In [8]:
count = {}
for k, v in orintation_count:
    count.setdefault(k, [0, 1])
    count[k][v] +=1

In [9]:
pgr.rc_map

{'A': 'T',
 'C': 'G',
 'G': 'C',
 'T': 'A',
 'N': 'N',
 'a': 'T',
 'c': 'G',
 'g': 'C',
 't': 'A'}

In [10]:
with open("/home/yw2245/project_pi_hc895/yw2245/projects/lpa_genotyping/data/openhgl/work/LPA_HGRP_v2_seq_full_length_oriented.fa", "w") as f:
    for k, v in count.items():
        _, sid = k
        n, s = sdb0.get_seq_by_id(sid), sdb0.get_seq_by_id(sid)
        rc = 0
        if v[1] > v[0]:
            s = pgr.rc(s)
            rc = 1
        print(">{}_{}".format(n, rc), file = f)
        print(s, file = f)

In [11]:
sdb0 = pgr.SeqDB("/home/yw2245/project_pi_hc895/yw2245/projects/lpa_genotyping/data/openhgl/work/LPA_HGRP_v2_seq_full_length_oriented.fa")

sdb0.load_sequences()
sdb0.build_shmmrs_parallel(80, 56, 3)

In [12]:
weight = {}
kmers = set()
for sid in sorted(sdb0.get_all_ids()):
    locs = sdb0.get_shmmer_locs_by_id(sid)
    s = sdb0.get_seq_by_id(sid)
    for i in range(len(locs)-1):
        p1 = locs[i]
        p2 = locs[i+1]
        m1 = s[p1-56:p1]
        m2 = s[p2-56:p2]
        kmers.add(m1)
        kmers.add(m2)
        weight.setdefault((m1, m2),0)
        weight[(m1, m2)]+=1
    
G = nx.DiGraph()
for m1, m2 in weight:
    w = weight[(m1,m2)]
    #G.add_edge(m1,m2, weight=w, penwidth=w**0.5*0.75)
    G.add_edge(m1,m2)

kmer_count = {}
with open("/home/yw2245/project_pi_hc895/yw2245/projects/lpa_genotyping/data/openhgl/work/kmer_in_graph_FL.txt","w") as f:
    for m in list(kmers):
        print(m, file=f)
        kmer_count[m] = 0

"""
with open("t4") as f:
    for r in f:
        r = r.strip()
        r = r.split()
        G.nodes[r[0]]["width"] = 1
        G.nodes[r[0]]["height"] = 1
        
        if int(r[1]) > 10:
            G.nodes[r[0]]["fillcolor"] = "red"
            G.nodes[r[0]]["style"]="filled"
            # G.nodes[r[0]]["width"] = float(r[1])/500
            # G.nodes[r[0]]["height"] = float(r[1])/500
"""

'\nwith open("t4") as f:\n    for r in f:\n        r = r.strip()\n        r = r.split()\n        G.nodes[r[0]]["width"] = 1\n        G.nodes[r[0]]["height"] = 1\n        \n        if int(r[1]) > 10:\n            G.nodes[r[0]]["fillcolor"] = "red"\n            G.nodes[r[0]]["style"]="filled"\n            # G.nodes[r[0]]["width"] = float(r[1])/500\n            # G.nodes[r[0]]["height"] = float(r[1])/500\n'

In [13]:
len(G.nodes()),len(G.edges())

(5521, 7479)

In [14]:
#!head kmer_in_graph_FL.txt
!head /home/yw2245/project_pi_hc895/yw2245/projects/lpa_genotyping/data/openhgl/work/kmer_in_graph_FL.txt

TATATATTATATATAATATATATAATATATTATATATTATATATAATATATATAAT
TTATATATAATATATATTATATATAATATAATATATAATATATATTATATATTATA
TATATTATATATAATATATATTATATATAATATAATATATAATATATATTATATAT
TAACGACAAAAGTTGTAATTAAACTATTCTTCTTCCCAAATACCCACGTCTAATGT
TAATATATATTATATATAATATAATATATAATATATATTATATATTATATAATATA
AACACTGCAGTCATACTGCCCTTGCTGTACTTGGACTAGGGAAGGAAAAAAGATCC
TGGTTGGTTTTCTCCCATCCATCTGCCTACAGGTAAGGGAAAGATAACGTTCGTAA
CAACTGATGGCCCTTCCCTACCCACCCTGGTAGCAGAAGACAAAGAGCATATAATC
ATAATATATATTATATATAATATATATAATATATTATATATTATATATAATATATA
ACACATCATGCTTATAGATTAAAAAATCCAATGTCACTTTTCACAAAACTGATCTT


In [ ]:
nx.write_gexf(G, "/home/yw2245/project_pi_hc895/yw2245/projects/lpa_genotyping/data/openhgl/work/test_pan_LPA_FL_tmp.gexf")